In [1]:
import numpy as np
from numpy import *
from random import shuffle
import math
import pandas as pd
from sklearn.metrics import r2_score

In [2]:
def data_generate(M,P_PDG,W,N,K,Noise):
    #W是原数据集,代表哪些代理之间有连接，即网络
    #N是W的行数，即代理的个数
    #代理人的博弈策略
    player = zeros((M+1,N))
    #原始博弈策略放在第一行，随机设置
    for i in range(N):
        if random.random()<=0.5:
            player[0,i] = 1 #1代表合作
        else:
            player[0,i] = 0 #0代表不合作
    #计算每个节点的收益
    #M轮博弈
    F = zeros((1,N))
    G = zeros((M,N))
    Aa = zeros((M,N))
    A = zeros((N,M,N))
    for t in range(M):
        for i in range(N):
            if player[t,i] == 1:
                s1 = array([[1],[0]])
            else:
                s1 = array([[0],[1]])
            for j in range(N):
                if player[t,j] == 0:
                    s2 = array([[0],[1]])
                else:
                    s2 = array([[1],[0]])
                F[0,j] = ((s1.T).dot(P_PDG)).dot(s2) #如果代理i和j连接，则代理i的收益为F[0,j]
            A[i,t,:] = F  #A:N*M*N   
            # F是三维矩阵A的第i页，第t行
            G[t,i] = F.dot(W[:,i]) #第t轮代理i的收益
        # update strategies
        for k in range(N):
            s=[i for i,x in enumerate(list(W[:,k])) if x>=1]     # 找出与代理k合作的代理的索引
            if len(s)!=0: # 如果有代理与代理k合作
                shuffle(s)
                P = 1/(1+math.e**((G[t,k]-G[t,s[0]])/K)) # 费米规则
                if random.random()<= P:
                    player[t+1,k] = player[t,s[0]]
                else:
                    player[t+1,k] = player[t,k]
            else: # 如果没有代理与代理k合作
                player[t+1,k] = player[t,k]
    # add noise for G
    for i in range (M):
        if random.random()<0.5:
            Aa[i,:] = G[i,:]+ Noise*random.random((1,N)) # m轮收益总矩阵：包含每一轮的收益M*N
        else:
            Aa[i,:] = G[i,:]
    return [A,Aa]

In [3]:
class Lasso_SPL():
    def __init__(self):
        pass
    
    def initialize_params(self, dims):
        # 定义参数初始化函数
        w = np.zeros((dims, 1))
        b = 0
        return w, b
 
    def sign(self, x):
        # 定义符号函数
        if x > 0:
            return 1
        elif x < 0:
            return -1
        else:
            return 0
        
    def l1_loss(self, X, y, w, b, alpha, v):
        # 定义lasso损失函数
        num_train = X.shape[0]
        num_feature = X.shape[1]
        
        y_hat = np.dot(X, w) + b
        loss = np.sum(((y_hat - y) ** 2)*v) / num_train + np.sum(alpha*abs(w))
        dw = np.dot((X*v).T, (y_hat - y)) / num_train + alpha*np.vectorize(self.sign)(w)
        db = np.sum((y_hat - y)*v) / num_train
        return y_hat, loss, dw, db
    
    def lasso_train(self, X, y, learning_rate, epochs,v):
        # 定义训练过程
        loss_list = []
        w, b = self.initialize_params(X.shape[1])
        for i in range(1, epochs):
            y_hat, loss, dw, db = self.l1_loss(X, y, w, b, 0.1,v)
            w += -learning_rate * dw
            b += -learning_rate * db
            loss_list.append(loss)
            
            #if i % 1000 == 0:
                #print('epoch %d loss %f' % (i, loss))
            
            params = {
                'w': w,
                'b': b
            }
            grads = {
                'dw': dw,
                'db': db
            }
        return loss, loss_list, params, grads

In [4]:
class Lasso():
    def __init__(self):
        pass
    
    def initialize_params(self, dims):
        # 定义参数初始化函数
        w = np.zeros((dims, 1))
        b = 0
        return w, b
 
    def sign(self, x):
        # 定义符号函数
        if x > 0:
            return 1
        elif x < 0:
            return -1
        else:
            return 0
        
    def l1_loss(self, X, y, w, b, alpha):
        # 定义lasso损失函数
        num_train = X.shape[0]
        num_feature = X.shape[1]
        
        y_hat = np.dot(X, w) + b
        loss = np.sum((y_hat - y) ** 2) / num_train + np.sum(alpha*abs(w))
        dw = np.dot(X.T, (y_hat - y)) / num_train + alpha*np.vectorize(self.sign)(w)
        db = np.sum((y_hat - y)) / num_train
        return y_hat, loss, dw, db
    
    def lasso_train(self, X, y, learning_rate, epochs):
        # 定义训练过程
        loss_list = []
        w, b = self.initialize_params(X.shape[1])
        for i in range(1, epochs):
            y_hat, loss, dw, db = self.l1_loss(X, y, w, b, 0.1)
            w += -learning_rate * dw
            b += -learning_rate * db
            loss_list.append(loss)
            
            #if i % 1000 == 0:
                #print('epoch %d loss %f' % (i, loss))
            
            params = {
                'w': w,
                'b': b
            }
            grads = {
                'dw': dw,
                'db': db
            }
        return loss, loss_list, params, grads

In [5]:
M = 10 #m轮更新
b = 1.2 #叛逃者的收益
K = 0.1
P_PDG = array([[1,0],[b,0]]) #2*2的收益矩阵
Noise = 10
W = np.loadtxt("dolphin.txt") 
T = W.shape[0] #全部社区内个体的总数量
SV = 6
# the part for generate EG data
y = zeros((SV*M,T))
AA = zeros((T,SV*M,T))
for i in range(SV): #产生6组A和Y的值
    [A, Aa] = data_generate(M,P_PDG,W,T,K,Noise)
    for j in range(M):
        y[i*M+j,:] =Aa[j,:]
        for k in range(T):
            AA[k,i*M+j,:] = A[k,j,:]

In [6]:
if __name__ == '__main__':
    lasso = Lasso()
    X=zeros((T,T))
    for j in range(T):
        loss, loss_list, params, grads = lasso.lasso_train(AA[j], y[:,j].reshape(-1,1), 0.01, 3000)
        w = params['w']
        for i in range(w.shape[0]):
                X[i,j]=w[i]
np.savetxt("X.txt", X,fmt='%f',delimiter=' ')

In [15]:
if __name__ == '__main__':
    lasso = Lasso_SPL()
    X=zeros((T,T))
    v = ones((SV*M,1))
    v1 = zeros((SV*M,1))
    Los = zeros((SV*M,T))
    losss = zeros((SV*M,1))
    lambda_0=1.5
    t=0
    while (t<7):
        for i in range(SV*M):
            if losss[i]<lambda_0:
                v[i]=1-losss[i]/lambda_0
            else:
                v[i]=0
        if (v==v1).all():
            break
        else:
            for j in range(T):
                loss, loss_list, params, grads = lasso.lasso_train(AA[j], y[:,j].reshape(-1,1), 0.01, 3000, v.reshape(-1,1))
                w = params['w']
                b = params['b']
                Lo =  abs((np.dot(AA[j], w) + b)-y[:,j].reshape(-1,1))
                for i in range(SV*M):
                    Los[i,j]=Lo[i]
                for i in range(w.shape[0]):
                    X[i,j]=w[i]
            for i in range(SV*M):
                losss[i]=np.mean(Los[i,:])
            t = t+1
            lambda_0 = lambda_0*1.25
            #lambda_0 = lambda_0 + np.mean(Los)
np.savetxt("X1.txt", X,fmt='%f',delimiter=' ')

In [16]:
v

array([[0.        ],
       [0.03428055],
       [0.89595948],
       [0.0536224 ],
       [0.138522  ],
       [0.84521039],
       [0.06690163],
       [0.06474973],
       [0.        ],
       [0.06053501],
       [0.08107882],
       [0.14918602],
       [0.91339836],
       [0.91361459],
       [0.89934897],
       [0.26029331],
       [0.89197741],
       [0.89197741],
       [0.89197741],
       [0.21232655],
       [0.17154329],
       [0.91475433],
       [0.91773609],
       [0.08568111],
       [0.19247763],
       [0.89197741],
       [0.89197741],
       [0.21978531],
       [0.89197741],
       [0.21731763],
       [0.        ],
       [0.        ],
       [0.07107437],
       [0.89510585],
       [0.        ],
       [0.90583617],
       [0.90259711],
       [0.89183183],
       [0.09976027],
       [0.89293541],
       [0.85968918],
       [0.90766211],
       [0.03797345],
       [0.91798574],
       [0.15400587],
       [0.91824733],
       [0.20740648],
       [0.120

In [28]:
if __name__ == '__main__':
    lasso = Lasso_SPL()
    X=zeros((T,T))
    v = ones((SV*M,T))
    v1 = zeros((SV*M,T))
    Los = zeros((SV*M,T))
    lambda_0=2
    t=0
    while (t<5):
        for j in range(T):
            for i in range(SV*M):
                if Los[i,j]<lambda_0:
                    v[i,j]=1-Los[i,j]/lambda_0
                else:
                    v[i,j]=0
        if (v==v1).all():
            break
        else:
            for j in range(T):
                loss, loss_list, params, grads = lasso.lasso_train(AA[j], y[:,j].reshape(-1,1), 0.01, 3000, v[:,j].reshape(-1,1))
                w = params['w']
                b = params['b']
                Lo =  abs((np.dot(AA[j], w) + b)-y[:,j].reshape(-1,1))
                for i in range(SV*M):
                    Los[i,j]=Lo[i]
                for i in range(w.shape[0]):
                    X[i,j]=w[i]
            t = t+1
            lambda_0 = lambda_0*1.5
            #lambda_0 = lambda_0 + np.mean(Los)
np.savetxt("X2.txt", X,fmt='%f',delimiter=' ')

In [29]:
v

array([[0.54252909, 0.95151647, 0.82159206, ..., 0.80629268, 0.17793334,
        0.55997109],
       [0.91320495, 0.77746513, 0.20302121, ..., 0.99602391, 0.50642558,
        0.99902401],
       [0.87369123, 0.89673361, 0.79880098, ..., 0.88073914, 0.76036999,
        0.96939919],
       ...,
       [0.96027064, 0.79621643, 0.84764449, ..., 0.87506686, 0.61150574,
        0.98879227],
       [0.97841332, 0.86101777, 0.72557315, ..., 0.80660567, 0.93367552,
        0.81559939],
       [0.67245805, 0.88362215, 0.95300201, ..., 0.73584962, 0.17074905,
        0.25935975]])